<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/logo.png?raw=true" alt="2023年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

リードアライメント、SNP検出
---

　前回、シーケンシングデータからゲノム配列の復元を試みました。元のゲノムをカバーするほどのゲノム配列を利用できると、種間でのゲノム配列比較や、種内でのさまざまなゲノム解析の基準配列（**参照配列、リファレンス配列、Reference sequence**）として利用できるようになります。

　この実習テキストでは、次世代シーケンシングデータ（ショートリードデータ）を使って、どのようにサンプル間の変異を検出するかをみていきます。また、次回勉強するQTL-seq解析法の基礎「バルクシーケンシング」を勉強していきます。

### サンプルデータのダウンロード

　この実習で使用するサンプルデータをダウンロードします。

- 参照配列:
  - 品種Aのゲノム配列（300bp）
- リード:
  - 品種Aのショートリード（20bp）
  - 品種Bのショートリード（20bp）
  - 品種Aと品種Bの交配F<small>1</small>世代1個体のショートリード（20bp）
  - 品種Aと品種Bの交配F<small>2</small>世代100個体のショートリード（20bp）


In [ ]:
%%bash
## サンプルファイルのダウンロード
wget -q -O 'alignment_dataset.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/scripts/alignment_dataset.sh
bash alignment_dataset.sh
## 表示
echo 'Downloaded dataset: genome/ reads/ bam/ ngs/'

### ソフトウェアのダウンロード

　今回使用するソフトウェアとその実行プログラムをダウンロードをします。

In [ ]:
%%bash
## ソフトウェアのインストール
wget -q -O 'alignment_softwares.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/scripts/alignment_softwares.sh
bash alignment_softwares.sh
## 確認
ls tools/

# 今回の勉強内容

1. リードアライメントとSNP検出を学ぶ
  1. 品種Aのリードアライメント
  1. 品種BのリードアライメントとSNP検出
  1. 交配F<small>1</small>個体のアライメントデータ
  1. 交配F<small>2</small>個体のアライメントデータ
  1. 形質と連関しているSNP箇所を探す
  1. シーケンシングの種類と特徴
1. バルクシーケンシングを学ぶ
  1. バルクシーケンシング
  1. 実例: イネのQTL-seq



# 1. リードアライメントとSNP検出を学ぶ



## 1-1. 品種Aのリードアライメント

　最初に、**リードアライメント（Read alignment）**について勉強していきましょう。リードアライメントは、なんらかの参照配列（通常は、同種の1個体のゲノム配列）に対して、シーケンシングデータ（**リード Reads**）を並べることを言います。その際、多少のミスマッチ（塩基の違い）は許容して、配列が似ている箇所にリードを並べます。

　リードアライメントをおこなうと、参照配列とリードの間の塩基の違いを容易に観察できるようになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/read_alignment.png?raw=true" alt="Read alignment" height="80px" align="middle">

<small>※1 リードアライメントの仕組みに興味がある方は以下のWebページを参照してください。<br>　[How Do We Compare Biological Sequences?](https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5) (Bioinformatics Algorithms / Phillip Compeau)</small>




　ここでは、以下2つの配列データを使います。
- 参照配列: 品種Aのゲノム配列（300bp）
  - /content/genome/CultivarA.fa
- リード: 品種Aのリード（合計約240本の**ペアエンドリード**<sup>※2</sup>、各20bp）
  - /content/reads/CultivarA_read1.fq
  - /content/reads/CultivarA_read2.fq

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/paired_end_read.png?raw=true" alt="DNA sequencing and paired-end read" height="230px" align="middle">

　実際のゲノム解析でも良く使われている[**BWA (Burrows-Wheeler Alignment)**](https://bio-bwa.sourceforge.net/)と呼ばれるソフトウェアで、リードアライメントをおこなってみましょう。

　次のプログラムを実行して、リードアライメントをおこなってください。

In [ ]:
%%bash
## リードアライメント: 品種Aのリードを、品種Aのゲノム配列にアライメントする

## リードアライメントのための事前準備（ゲノム配列のインデックス作成）
if [ ! -e genome/CultivarA.fa.bwt ]; then bwa index genome/CultivarA.fa; fi

## リードアライメント
bwa aln genome/CultivarA.fa reads/CultivarA_read1.fq > read1.sai 2>/dev/null
bwa aln genome/CultivarA.fa reads/CultivarA_read2.fq > read2.sai 2>/dev/null
bwa sampe genome/CultivarA.fa read1.sai read2.sai reads/CultivarA_read1.fq reads/CultivarA_read2.fq > CultivarA.sam
samtools sort -O bam CultivarA.sam > CultivarA.bam
samtools index CultivarA.bam
rm -fR read1.sai read2.sai

　アライメント結果を、[**IGV (Integrative Genomics Viewer)**](https://software.broadinstitute.org/software/igv/home)で確認してみましょう。

In [ ]:
## アライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
A = BamTrack({ "name":"CultivarA", "path":"CultivarA.bam", "indexPath":"CultivarA.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
b.load_track(A)

　一部のリード内には、参照配列と異なる塩基が見つかってきます。その異なる塩基には二つの可能性があります。
- シーケンシングしたサンプル内にみられる変異（**一塩基多型; Single nucleotide polymorphism, SNP**）
- シーケンシング中に発生する塩基の読み間違い（シーケンシングエラー）

　じゅうぶんな数のリードがあれば、その異なる塩基がSNPなのかシーケンシングエラーなのかを判別できます。

　今回、品種Aのリードはじゅうぶん多数あります。「参照配列と異なる塩基」は大部分のリードが共通にもっているわけではありません。そのため、見つかった「参照配列と異なる塩基」は、シーケンシングエラーと考えるのがよさそうです。

## 1-2. 品種BのリードアライメントとSNP検出

　今度は、品種Bのリードを品種Aの参照配列に対してアライメントしてみましょう。

使用するデータ:
- 参照配列: 品種Aのゲノム配列（300bp）
  - /content/genome/CultivarA.fa
- リード配列: 品種Bのリード配列（合計約240本のペアエンドリード、各20bp）
  - /content/reads/CultivarB_read1.fq
  - /content/reads/CultivarB_read2.fq

In [ ]:
%%bash
## リードアライメント: 品種Bのリードを、品種Aのゲノム配列にアライメントする

## リードアライメントの事前準備
if [ ! -e genome/CultivarA.fa.bwt ]; then bwa index genome/CultivarA.fa; fi
## リードアライメント
bwa aln genome/CultivarA.fa reads/CultivarB_read1.fq > read1.sai 2>/dev/null
bwa aln genome/CultivarA.fa reads/CultivarB_read2.fq > read2.sai 2>/dev/null
bwa sampe genome/CultivarA.fa read1.sai read2.sai reads/CultivarB_read1.fq reads/CultivarB_read2.fq > CultivarB.sam
samtools sort -O bam CultivarB.sam > CultivarB.bam
samtools index CultivarB.bam
rm -fR read1.sai read2.sai

In [ ]:
## アライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
B = BamTrack({ "name":"CultivarB", "path":"CultivarB.bam", "indexPath":"CultivarB.bam.bai", "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
b.load_track(B)

　アライメントの結果、一部の塩基箇所において、ほぼすべてのリードが異なる塩基を共通にもっています。その塩基の違いは、シーケンシングエラーではなく、品種Aと品種Bの間のSNP変異と考えられます。

## 1-3. 交配F<small>1</small>世代のアライメントデータ

　品種Aと品種Bを人為的に交配して得られた個体（F<small>1</small>個体）では、どのようなリードアライメント結果が得られるか見てみましょう。以下を実行すると、F<small>1</small>個体のリードアライメント結果を表示できます。

<small>※ ここからは、bwaによるリードアライメントを省略し、事前に用意したアライメント結果を見ていきます。</small>

In [ ]:
## アライメント結果を見る
## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
B = BamTrack({ "name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai", "viewAsPairs":True, "displayMode":"SQUISHED", "height":150 })
F1= BamTrack({ "name":"F1",       "path":"bam/F1.bam",       "indexPath":"bam/F1.bam.bai",       "viewAsPairs":True })
## IGV表示
b = igv_notebook.Browser(ref)
b.load_track(B)
b.load_track(F1)

　F<small>1</small>個体は、品種Aと品種Bから一つずつ配偶子を受け取って生じた個体であり、それぞれの品種に由来するSNP塩基（以下、**SNPアリル、SNP allele**）をもっているはずです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/f1_hybrid.png?raw=true" alt="SNP allele heterozygous of F1" height="200px" align="middle">

　アライメント結果の各SNP箇所の塩基構成をみると、多少ばらついているものの、二つのSNPアリルがおよそ50%の割合で存在しています。

　このように、どちらかのSNPアリルに固定していない状態を、**ヘテロ接合型（Heterozygote）**と言います。一方で、品種Aや品種Bにみられるように、どちらかのSNPアリルに固定された状態を、**ホモ接合型（Homozygote）**と言います。SNPアリルの保有状況から遺伝子型（ヘテロ接合型かホモ接合型か）を決めることを**ジェノタイピング（Genotyping）**といいます。




## 1-4. 交配F<small>2</small>世代のアライメントデータ

　F<small>1</small>個体を自殖し、F<small>2</small>世代の子孫を得ると、SNPアリルは各個体にランダムに遺伝されます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/SNP_allele_segregation.png?raw=true" alt="SNP allele segregation in F2" height="230px" align="middle">

　F<small>2</small>世代では各個体のリードアライメントがどのようになるか見てみましょう。次のプログラムを実行すると、F<small>2</small>世代の100個体のうち、指定のF<small>2</small>個体（例えば、F2_001からF2_005）のアライメント結果を見れます。

In [ ]:
## 表示するF2番号: 1~100までの数字を自由に組み合わせ可能
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2 = [1,2,3,4,5]

## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
Info = [BamTrack({ **{"name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"F1",       "path":"bam/F1.bam", "indexPath":"bam/F1.bam.bai"}, **bam_common_info })]
for f in sorted(list(set(F2))):
    Info += [ BamTrack({ **{"name":"F2_{:03}".format(f),
                          "path":"/content/bam/F2_{:03}.bam".format(f),
                          "indexPath":"/content/bam/F2_{:03}.bam.bai".format(f)}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)

　F<small>2</small>世代の個体ごとに各SNP箇所のSNPアリルの構成が異なっており、SNPアリルの分離を確認できました。

　F<small>2</small>個体集団のように、SNPアリルがその場所ごとに独立に分離した（染色体がシャッフルされた）集団は、両親間の形質の違いを生み出す遺伝子（原因遺伝子）のマッピングに役立ちます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/chromosome_shuffling.png?raw=true" alt="Chromosome shuffling" height="230px" align="middle">







## 1-5. 形質と連関しているSNP箇所を探す
　つぎは、形質情報も考慮し、リードアライメント結果を見ていきましょう。各個体の形質データは次のとおりです。
- F<small>2</small>各個体の形質: [形質データ表](https://github.com/CropEvol/lecture/blob/master/textbook_2023/dataset/f2_phenotypes.pdf)
- 品種Aの形質: Short
- 品種Bの形質: Long
- F<small>1</small>個体の形質: Medium


　品種Aと同じ「Short」形質をもつF<small>2</small>個体を4つ程度選び、それらのアライメント結果を表示してください。さらに、そのグループ内で分離していないSNP（固定SNP）を探してみてください。


In [ ]:
## 品種Aと同じShort形質をもつF2個体番号を半角数字で入力してください。
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2_S_bulk = []

## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
Info = [BamTrack({ **{"name":"CultivarA", "path":"bam/CultivarA.bam", "indexPath":"bam/CultivarA.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai"}, **bam_common_info }),]
for f in sorted(list(set(F2_S_bulk))):
    Info += [ BamTrack({ **{"name":"F2_{:03}".format(f),
                          "path":"bam/F2_{:03}.bam".format(f),
                          "indexPath":"bam/F2_{:03}.bam.bai".format(f)}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)

　同様に、品種Bと同じ「Long」形質のF<small>2</small>個体についても、4個体分程度のアライメント結果を表示し、固定SNPを探してください。

In [ ]:
## 品種Bと同じLong形質をもつF2個体番号を半角数字で入力してください。
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2_L_bulk = []

## 準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
## 準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## 準備: Reads info
Info = [BamTrack({ **{"name":"CultivarA", "path":"bam/CultivarA.bam", "indexPath":"bam/CultivarA.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"CultivarB", "path":"bam/CultivarB.bam", "indexPath":"bam/CultivarB.bam.bai"}, **bam_common_info }),]
for f in sorted(list(set(F2_L_bulk))):
    Info += [ BamTrack({ **{"name":"F2_{:03}".format(f),
                          "path":"bam/F2_{:03}.bam".format(f),
                          "indexPath":"bam/F2_{:03}.bam.bai".format(f)}, **bam_common_info }) ]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)

　形質ごとに4個体程度を表示し、各SNP箇所のSNPアリル構成を見ていくと、前から55番目の塩基に固定SNPを発見できます。

55番目の塩基:
- Short形質の品種Aでは「A」
- Short形質のF<small>2</small>個体でも「A」のみ（品種A由来のアリルのホモ接合）
- Long形質の品種Bでは「T」
- Long形質のF<small>2</small>個体でも「T」のみ（品種B由来のアリルのホモ接合）

　形質とSNPアリルに連関（Association）がみられることから、形質の違いを作っている原因変異かもしれません。

<small>※ 「Medium」形質のF<small>2</small>個体のアライメント結果には、55番塩基に2つのSNPアリルが見つかるはずです。余裕があれば確認してみてください。</small>


## 1-6. シーケンシングの種類と特徴

　各個体の遺伝子型決定（ジェノタイピング）によく使われるシーケンシング方法について勉強しましょう。
- 全ゲノムシーケンシング（Whole genome sequencing: WGS）
- RNAシーケンシング（RNA sequencing: RNA-seq）
- RADシーケンシング（RAD sequencing: RAD-seq）


In [ ]:
## 準備: ライブラリなど
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
igv_notebook.init()
base = "/content/ngs/"
bam_common_info = { "showCoverage":False, "displayMode":"SQUISHED", "viewAsPairs":True, "height":100 }

## 準備: Reference info
ref = RefTrack({ "fastaPath":base+"Ref.genome.fa", "indexPath":base+"Ref.genome.fa.fai", "id":"Reference" }, locus="chr01:10,000-35,000")
genes = AnnotationTrack({ "name":"Genes", "path":base+"Ref.transcript.gff", "format":"gff", "displayMode":"SQUISHED" })
restsite= AnnotationTrack({ "name":"Restriction site", "path":base+"Ref.radseq_enzymes.bed", "format":"bed", "color":"#ff7f0e"})
## 準備: Reads info
Info = [BamTrack({ **{"name":"WGS", "path":base+"wgs.bam", "indexPath":base+"wgs.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"RNA-seq", "path":base+"rnaseq.bam", "indexPath":base+"rnaseq.bam.bai"}, **bam_common_info }),
        BamTrack({ **{"name":"RAD-seq", "path":base+"radseq.bam", "indexPath":base+"radseq.bam.bai"}, **bam_common_info }),]
## IGV表示
b = igv_notebook.Browser(ref)
for i in Info: b.load_track(i)
b.load_track(genes)
b.load_track(restsite)

　それぞれのシーケンシング対象と得られるSNP、特徴をまとめると次のとおりです。

| 種類 | シーケンシング対象領域 / SNP探索領域 | 特徴 |
|:---|:---|:---|
| Whole genome sequencing | ゲノム全配列 　| 解析には多くのリードを必要とする。 |
| RNA sequencing | 発現遺伝子（mRNA）配列<br>（イントロンは含まない）　| 得られるリード量は遺伝子発現量に依存する。<br>必要なリード量もWGSに比べて少なく済む。 |
| RAD sequencing | 任意の制限酵素切断部位近くの配列　| 少ないリード量で解析をおこなえる。<br>類似シーケンシング手法: GBS, GRAS-Di, MIG-seq |


# 2. バルクシーケンシングを学ぶ

## 2-1. バルクシーケンシング

　先ほどは、各個体のアライメント結果をサンプルごとに表示しました。**バルクシーケンシング**のリードは、いくつかの個体のリードをランダムにいくつか集めてプーリングし、"1サンプル分"のデータとして扱ったようなものとほぼ同じです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/read_sampling.png?raw=true" alt="Bulked DNA sequencing" height="250px" align="middle">

　次のコードセルの`F2_S_bulk`（Short形質グループ: S-bulk）と`F2_L_bulk`（Long形質グループ: L-bulk）の右辺の括弧`[]`内に、4個体程度の個体番号を入力して、バルクシーケンシングリードのアライメント結果を見てみましょう。


In [ ]:
## プーリングするF2個体の番号
## 注意： 数字と数字の間には半角のカンマ(,)が必要です。
F2_S_bulk = []
F2_L_bulk = []

## 以下は、変更不要
## Pooling
import re
def Pooling(Ns, bam_dir, samplingRate=-1, output="bulk.bam", verbose=False, samtools_path="samtools"):
    f2_list = [bam_dir+"/F2_{:03}.bam".format(f) for f in sorted(list(set(Ns)))]
    if samplingRate>=1.0:
        samplingRate = 1.0
    elif samplingRate<=0.0:
        samplingRate = float(1.0 / len(bam_dir) *2)
    f2_str = " ".join(f2_list)
    # リードのサンプリング
    !$samtools_path merge -o - $f2_str | samtools view -b -s $samplingRate > $output
    !$samtools_path index $output
    if verbose:
      f2_samples = [re.search(r".*/(.+?)$", i).group(1) for i in f2_list]
      print("Output: ", output)
      print("- Reads sampled from: "+",".join(f2_samples))
      print("- Sampling rate: %.3f" % samplingRate)

Pooling(F2_S_bulk, bam_dir="bam/", output="S_bulk.bam", verbose=True)
Pooling(F2_L_bulk, bam_dir="bam/", output="L_bulk.bam", verbose=True)

## IGV表示の準備: ライブラリ
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
## IGV表示の準備: Reference info
ref = RefTrack({ "fastaPath":"genome/CultivarA.fa", "indexPath":"genome/CultivarA.fa.fai", "id":"CultivarA" })
## IGV表示の準備: Reads info
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":200 }
S = BamTrack({ **{"name":"S-bulk", "path":"S_bulk.bam", "indexPath":"S_bulk.bam.bai"}, **bam_common_info })
L = BamTrack({ **{"name":"L-bulk", "path":"L_bulk.bam", "indexPath":"L_bulk.bam.bai"}, **bam_common_info })
## IGV表示
igv_notebook.init()
b = igv_notebook.Browser(ref)
b.load_track(S)
b.load_track(L)

### 実際のバルクシーケンシングリード

　今回、指定した個体のシーケンシングリードをランダムに集めてバルクシーケンシングリードを用意しました。しかし、実際のバルクシーケンシングでは少し異なります。

　実際のバルクシーケンシングでは、DNAの段階で形質ごとにDNAをプーリングしてシーケンシングします。そのため、プーリングしたどれかの個体のDNAから由来したリードであるのは間違いないですが、どの個体由来のリードかの特定はできません。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/bulk_sequencing.png?raw=true" alt="bulk sequencing" height="280px" align="middle">



　バルクシーケンシングの利点は、なんらかの形質に関わる遺伝子のマッピングをおこなう際、分離集団（例えば、F<small>2</small>個体集団）の各個体を個別にシーケンシングするよりも費用を抑えられるためです。

例）シーケンシング費用が1サンプル10,000円の場合:  
- F<small>2</small>集団100個体の個別シーケンシング: 1,000,000円
- F<small>2</small>集団の二つの形質のバルクシーケンシング: 20,000円<sup>※3</sup>

<small>※3 QTL-seqでは形質ごとに数サンプル〜数十サンプル分に相当するデータ量が必要になるため、実際にはもう少しかかります。</small>

## 2-2. 実例: イネのバルクシーケンシング解析

　イネのバルクシーケンシング解析の実例を見てみましょう。

　イネいもち病にかかりにくい品種（抵抗性品種）とかかりやすい品種（罹病性品種）の交配からできた分離集団で、抵抗性個体グループと罹病性個体グループをバルクシーケンシングし、罹病性品種のゲノム配列にアライメントしたデータです。

- 参照配列: 罹病性品種のゲノム配列
- バルクシーケンシングリード:
  - 抵抗性個体のバルクシーケンシングリード
  - 罹病性個体のバルクシーケンシングリード

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_takagi_etal_2013_2.png?raw=true" alt="Rice QTL-seq example (Takagi et al., 2013)" height="250px" align="middle">





　次の2つのコードセルを実行すると、イネのアライメント結果（イネ6番染色体の15Mbのみ）を見れます。アライメント結果から、形質と連関がありそうなSNP箇所（または、ゲノム領域）を探してみてください。




In [ ]:
## Colab上にデータファイルのダウンロードする
!pip install -q -U igv-notebook==0.3.1
!wget -q -O igv_prep.py    https://github.com/CropEvol/lecture/raw/master/textbook_2022/scripts/igv_prep.py
!wget -q -O rice_chr6_1_15Mb.fa    https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/rice_chr6_1_15Mb.fa
!wget -q -O rice_chr6_1_15Mb.fa.fai https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/rice_chr6_1_15Mb.fa.fai
!wget -q -O rice_R_bulk.bam    https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/R_bulk.sample.bam
!wget -q -O rice_R_bulk.bam.bai https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/R_bulk.sample.bam.bai
!wget -q -O rice_S_bulk.bam    https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/S_bulk.sample.bam
!wget -q -O rice_S_bulk.bam.bai https://raw.githubusercontent.com/qqep685d/sample_dataset/main/qtlseq/S_bulk.sample.bam.bai

In [ ]:
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
bam_common_info = { "displayMode":"SQUISHED", "alleleFreqThreshold":0.4, "height":200 }

## 準備: Reference info
ref    = RefTrack({ "fastaPath":"rice_chr6_1_15Mb.fa", "indexPath":"rice_chr6_1_15Mb.fa.fai", "id":"Rice" }, locus="chr6:1-10,000")
## 準備: Reads info
r_bulk = BamTrack({ **{"name":"R_bulk", "path":"rice_R_bulk.bam", "indexPath":"rice_R_bulk.bam.bai"}, **bam_common_info })
s_bulk = BamTrack({ **{"name":"S_bulk", "path":"rice_S_bulk.bam", "indexPath":"rice_S_bulk.bam.bai"}, **bam_common_info })
## IGV表示
igv_notebook.init()
b = igv_notebook.Browser(ref)
b.load_track(r_bulk)
b.load_track(s_bulk)

　イネの染色体は12本あります。今回表示したデータは、イネ6番染色体の一部領域のみ（chr6:1-15,000,000）です。この領域に形質と連関するSNP箇所が含まれていますが、このアライメント結果では多量のSNPがあり、目視でそのSNP箇所を探すのは大変です。

　次回の実習で、SNP情報をPythonプログラミングで解析し、原因遺伝子があるゲノム領域の探索をしていきましょう。

# まとめ

　今回、リードアライメントとSNP検出、バルクシーケンシングについて勉強しました。
- リードアライメント
  - 似た配列の場所を参照配列から探し、参照配列にリードを並べる
- SNP検出
  - 参照配列とリード配列を比較し、信頼性のある塩基の違い（SNP）を探す
- バルクシーケンシング
  - 複数個体のDNAをプーリングし、そのバルクDNAをシーケンシングする
  - とくに、遺伝子マッピングでは、形質ごとにバルクDNAを用意し、バルクシーケンシングをおこなう

  


<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>